## pytorch_seresnext101-32x4d+kfold+focal loss v3.0


* fold = 4
* epoch = 3
* γ=0.4

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from PIL import Image, ImageOps, ImageFilter
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import random
import datetime
import os

import cv2  
from sklearn import preprocessing 
from sklearn.model_selection import KFold
import h5py

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose,RandomContrast,RandomBrightness,Resize
)
import albumentations

In [3]:
from torch.utils.data import DataLoader
from prefetch_generator import BackgroundGenerator

class DataLoaderX(DataLoader):

    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

In [4]:
# os.chdir("../input/pretrained_PyTorch/")
path = os.getcwd()
print(path)

/home/user1/Downloads/iMet-Collection-2019-FGVC6


In [5]:
train = pd.read_csv(path+"/train.csv")
lable = pd.read_csv(path+"/labels.csv")
test = pd.read_csv(path+"/sample_submission.csv")

In [6]:
lable_length = len(lable)
train_length = len(train)
test_length = len(test)
print(train_length)
print(lable_length)
print(test_length)

109237
1103
38801


In [7]:
print(np.array(lable)[397])
print(np.array(lable)[398])
c_length = len(np.array(lable)[:398])
t_length = len(np.array(lable)[398:])
print(c_length)
print(t_length)

[397 'culture::zurich']
[398 'tag::abbies']
398
705


In [8]:
def augment(p=.5):
    return Compose([
        HorizontalFlip(p=0.5),
        OneOf([
            RandomBrightness(0.1, p=1),
            RandomContrast(0.1, p=1),
        ], p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
        IAAAdditiveGaussianNoise(p=0.3)
        #Resize(220,220)
    ], p=p)

aug = augment(0.2)

In [9]:
def creatData(train,lable_length):
    train = np.array(train)
    trainA_data = []
    lab_data = []
    #trainC_data = []
    #trainT_data = []
    for t in range(train_length):
        v = np.zeros(lable_length)
        #print(train[t,1])
        lab = []
        for s in train[t,1].split(" "):
            #print(s)
            v[int(s)] = 1
            lab.append(int(s))
        trainA_data.append([train[t,0],v])
        lab_data.append([train[t,0],np.array(lab)])
    return np.array(trainA_data),np.array(lab_data)

In [10]:
train_a,train_lab = creatData(train,lable_length)

In [11]:
train_lab[0]

array(['1000483014d91860', array([147, 616, 813])], dtype=object)

In [12]:
def dfilter(data,th):
    dfilter = []
    for i in range(len(data)):
        if train_a[i][1].sum()>th:
            dfilter.append(train_a[i])
    return np.array(dfilter)

In [13]:
datafilter = dfilter(train_a,1) # remain feature great then 1

In [14]:
print("amount of image before: {}".format(len(train_a)))

print("amount of image after: {}".format(len(datafilter)))

amount of image before: 109237
amount of image after: 104913


In [15]:
# not use
datafilter = train_a

In [16]:
datafilter[0]

array(['1000483014d91860', array([0., 0., 0., ..., 0., 0., 0.])],
      dtype=object)

In [17]:
image_resize = 220
data_transforms2 = transforms.Compose([
    transforms.Resize((image_resize,image_resize)),
    #transforms.RandomResizedCrop(250),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ])


data_transforms = transforms.Compose([
    transforms.Resize((image_resize,image_resize)),
    #transforms.RandomResizedCrop(250),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(
    #        [0.485, 0.456, 0.406], 
    #        [0.229, 0.224, 0.225])
    ])

train_transformer = transforms.Compose([
    transforms.Resize((128,128)),              # resize the image to 
    #transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.ToTensor(),           # transform it into a PyTorch Tensor
    #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
])

In [18]:
# top_transforms = transforms.Compose([
#     augment()['image'],
#     transforms.Resize((image_resize,image_resize)),
#     HorizontalFlip(p=0.5),
#     OneOf([
#         RandomBrightness(0.1, p=1),
#         RandomContrast(0.1, p=1),
#     ], p=0.3),
#     ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
#     IAAAdditiveGaussianNoise(p=0.3),
#     #transforms.RandomResizedCrop(250),
#     #transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(
#             [0.485, 0.456, 0.406], 
#             [0.229, 0.224, 0.225])
#     ])

In [19]:
def load2mem(in_file,transform):
    print("Load data from .h5 file...")
    t = time.time()
    prefile = h5py.File(in_file, 'r')
    preloadimg = []
    preloadleb = prefile['train_labels'][:,:][:20000]
    #print(prefile['train_img'][0,:,:])
#     for i in range(len(preloadleb)):
#         img = Image.fromarray(prefile['train_img'][i,:,:])
#         preloadimg.append(img)
    preloadimg = prefile['train_img'][:,:,:][:20000]
    
    print("Done")
    print("Take",datetime.timedelta(seconds = int(time.time()-t)),"seconds")
    return preloadimg, preloadleb


In [20]:
#a,b = load2mem("all_data_109237v1.h5", data_transforms2)

In [21]:
class dataset_h5(torch.utils.data.Dataset):
    def __init__(self, in_file, transform, kdata, augment = None,  mode = 'train'):
        super(dataset_h5, self).__init__()
        print("mode:",mode)
        self.file = h5py.File(in_file, 'r')
        self.n_images = len(kdata)
        self.kdata = kdata
        self.transform = transform
        self.augment = augment
        print("Load data from .h5 file...")
        t = time.time()
        if 'train' in format(mode):
            self.preloadimg = self.file['train_img'][self.kdata,:,:]
            self.preloadleb = self.file['train_labels'][self.kdata,:]
        elif 'test' in format(mode):
            self.preloadimg = self.file['test_img'][self.kdata,:,:]
            self.preloadleb = self.file['test_labels'][self.kdata]
        print("Take",datetime.timedelta(seconds = int(time.time()-t)),"seconds")

    def __getitem__(self, index):
        input_img = Image.fromarray(self.preloadimg[index])
        
        if self.augment is not None:
            #PIL to openCV
            imgv = cv2.cvtColor(np.asarray(input_img),cv2.COLOR_RGB2BGR)
            imgv = self.augment(image = imgv)['image']
            #openCV to PIL
            input_img = Image.fromarray(cv2.cvtColor(imgv,cv2.COLOR_BGR2RGB))
        
        input_img = self.transform(input_img)
        input_lab = self.preloadleb[index]
        return input_img,input_lab

    def __len__(self):
        return self.n_images

In [22]:
score_dataloader = DataLoader(dataset_h5("all_data_109237v1.h5", data_transforms2, [i for i in range(1000)]),
                              batch_size=80, 
                              shuffle=False,
                              num_workers=16, 
                              pin_memory=True)

mode: train
Load data from .h5 file...
Take 0:00:00 seconds


In [23]:
test_dataloader = DataLoader(dataset_h5("all_data_test_38801v1.h5", data_transforms2, [i for i in range(test_length)],mode='test'),
                             batch_size=80,
                             shuffle=False,
                             num_workers=16, 
                             pin_memory=True)

mode: test
Load data from .h5 file...
Take 0:00:01 seconds


In [24]:
# plt.figure(figsize=(10,10))
# plt.subplot(2,2,1)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[15][0]).convert('RGB'))
# plt.subplot(2,2,2)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[15][1]).convert('RGB'))
# plt.subplot(2,2,3)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[29][0]).convert('RGB'))
# plt.subplot(2,2,4)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[29][1]).convert('RGB'))


In [25]:
#(np.array(trainC_dataloader.dataset[4][0]).shape)

In [26]:
import pretrainedmodels

model = pretrainedmodels.__dict__["se_resnext101_32x4d"](num_classes=1000, pretrained='imagenet')
model.last_linear = nn.Linear(in_features=2048, out_features=lable_length, bias=True)

if torch.cuda.device_count() > 1:
    # device_ids has a default : all
    model = torch.nn.DataParallel(model, device_ids=[0,1]) 

model.to(device)

DataParallel(
  (module): SENet(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [27]:
from torchsummary import summary
summary(model, (3, 220, 220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 110, 110]           9,408
       BatchNorm2d-2         [-1, 64, 110, 110]             128
              ReLU-3         [-1, 64, 110, 110]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
            Conv2d-5          [-1, 128, 55, 55]           8,192
            Conv2d-6         [-1, 64, 110, 110]           9,408
       BatchNorm2d-7          [-1, 128, 55, 55]             256
              ReLU-8          [-1, 128, 55, 55]               0
       BatchNorm2d-9         [-1, 64, 110, 110]             128
             ReLU-10         [-1, 64, 110, 110]               0
        MaxPool2d-11           [-1, 64, 55, 55]               0
           Conv2d-12          [-1, 128, 55, 55]           8,192
      BatchNorm2d-13          [-1, 128, 55, 55]             256
             ReLU-14          [-1, 128,

In [28]:
model = torch.load('net_se_resnext101_t2.pkl')

In [29]:
def findPre(output,gate):
    a = ''
    output = np.array(output)
    m = np.max(output)

    for i in range(len(output)):
        #s = np.where(v[i] > 0.95, 1, 0)
        if output[i]>gate:
            #print(output[i])
            a = a + format(i)+' '
            
    #print(a)
    return a
    

In [36]:
def test(model,dloader,threshold):
    model.eval().cuda()
    #lengthD = len(dloader.dataset)
    ans = []
    out = []
    for step, (x, y) in enumerate(dloader):
        data = Variable(x).cuda()
        target = y
        with torch.no_grad():
            output = model(data).detach()
        
        v = output.cpu()
        v = torch.sigmoid(v)
        v = np.array(v)
        v = preprocessing.minmax_scale(v, feature_range=(0,1),axis=1)
        for i in range(len(v)):
            out.append(np.where(v[i] > threshold, 1, 0))
            s = findPre(v[i],threshold)
            #tar = str(target[i].numpy(), encoding = "utf-8")
            tar = str(target[i], encoding = "utf-8")
            ans.append([tar,s])
        if step %10 == 0:
            print('[{}/{} ({:.1f}%)]'.format(step * len(data), 
                                        len(dloader.dataset),
                                        100.*step/len(dloader)))
            
        data.cpu()
        torch.cuda.empty_cache()
    print("Finish")
    return ans,out
    

In [31]:
from sklearn.metrics import fbeta_score

def makeScore(pre,ans):
    pre = np.array(pre)
    va = fbeta_score(y_pred=pre, y_true=ans, beta=2, average="samples")
    print("Score : {:.5f}".format(va))
    return va
    

In [32]:
def findThreshold():
    score = []
    candidates = np.arange(0, 1.0, 0.01)
    for th in candidates:
        print("Threshold : {:.2f}".format(th))
        _,pre = test(model = model,dloader = score_dataloader,threshold = th)
        #return pre
        score.append(makeScore(np.array(pre),np.array(train_a[:1000,1].tolist())))
        print("=============================")
    pm = np.array(score).argmax()
    best_th, best_score = candidates[pm], score[pm]
    return best_th, best_score

In [33]:
bt, bs = findThreshold()
print("Best Threshold : {:.2f}".format(bt))
print("Best Score : {:.5f}".format(bs))

Threshold : 0.00
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.01418
Threshold : 0.01
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.24974
Threshold : 0.02
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.34709
Threshold : 0.03
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.41380
Threshold : 0.04
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.46657
Threshold : 0.05
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.50737
Threshold : 0.06
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.54207
Threshold : 0.07
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.56846
Threshold : 0.08
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.59464
Threshold : 0.09
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.61297
Threshold : 0.10
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.63047
Threshold : 0.11
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.64501
Threshold : 0.12
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.65736
Threshold : 0.13
[0/1000 

[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.53527
Threshold : 0.79
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.53124
Threshold : 0.80
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.52643
Threshold : 0.81
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.51835
Threshold : 0.82
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.51066
Threshold : 0.83
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.50710
Threshold : 0.84
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.50363
Threshold : 0.85
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.49731
Threshold : 0.86
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.48980
Threshold : 0.87
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.48166
Threshold : 0.88
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.47785
Threshold : 0.89
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.47352
Threshold : 0.90
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.46768
Threshold : 0.91
[0/1000 (0.0%)]
[800/1000

In [34]:
torch.cuda.empty_cache()
!nvidia-smi

Tue Nov 26 15:31:36 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0  On |                  N/A |
| 80%   68C    P2    79W / 250W |   1353MiB / 11175MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 40%   67C    P2    75W / 250W |    611MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [37]:
sub,_ = test(model = model,dloader = test_dataloader,threshold = bt)

[0/38801 (0.0%)]
[800/38801 (2.1%)]
[1600/38801 (4.1%)]
[2400/38801 (6.2%)]
[3200/38801 (8.2%)]
[4000/38801 (10.3%)]
[4800/38801 (12.3%)]
[5600/38801 (14.4%)]
[6400/38801 (16.5%)]
[7200/38801 (18.5%)]
[8000/38801 (20.6%)]
[8800/38801 (22.6%)]
[9600/38801 (24.7%)]
[10400/38801 (26.7%)]
[11200/38801 (28.8%)]
[12000/38801 (30.9%)]
[12800/38801 (32.9%)]
[13600/38801 (35.0%)]
[14400/38801 (37.0%)]
[15200/38801 (39.1%)]
[16000/38801 (41.2%)]
[16800/38801 (43.2%)]
[17600/38801 (45.3%)]
[18400/38801 (47.3%)]
[19200/38801 (49.4%)]
[20000/38801 (51.4%)]
[20800/38801 (53.5%)]
[21600/38801 (55.6%)]
[22400/38801 (57.6%)]
[23200/38801 (59.7%)]
[24000/38801 (61.7%)]
[24800/38801 (63.8%)]
[25600/38801 (65.8%)]
[26400/38801 (67.9%)]
[27200/38801 (70.0%)]
[28000/38801 (72.0%)]
[28800/38801 (74.1%)]
[29600/38801 (76.1%)]
[30400/38801 (78.2%)]
[31200/38801 (80.2%)]
[32000/38801 (82.3%)]
[32800/38801 (84.4%)]
[33600/38801 (86.4%)]
[34400/38801 (88.5%)]
[35200/38801 (90.5%)]
[36000/38801 (92.6%)]
[36800/388

In [ ]:
print(sub[:5])

In [38]:
sub =  pd.DataFrame(sub)

In [39]:
sub = sub.rename(index=str, columns={0: "id", 1: "attribute_ids"})

In [40]:
sub.head

<bound method NDFrame.head of                      id                                      attribute_ids
0      1002e7c8e9172e5c                          79 477 584 813 1062 1092 
1      10045628394b0c85                               51 718 813 903 1092 
2      1006c05aacbeaea6                   147 194 477 485 639 671 780 950 
3      10073dbee72a9e88                                   121 182 464 754 
4      10077d92747afb1a                       156 189 462 580 733 734 993 
5      1007e79b1424bcf4                            51 156 477 639 744 950 
6      100964e6d2a3fcb8                                13 554 670 813 896 
7       100b7c502f72a6f                                  189 194 671 1034 
8      100c3d33ff4a3ec7                                  156 541 813 1092 
9      100e371d42878eda                                       121 432 724 
10     1010d8a46bf001db                      13 14 51 127 671 675 780 978 
11     10135b09d753cb8b                                13 51 147 716 1

In [41]:
sub.to_csv('submission_t2.csv', index=False)